<a href="https://colab.research.google.com/github/shinkangecon/quantecon_python/blob/master/Aiyagari_EGM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Intitial Setup**

In [ ]:
# Practice for Aiyagari model
# Initial Setup
import numpy as np
import numba
from scipy import optimize
import matplotlib.pyplot as plt

# some useful plot defaults
plt.rcParams.update({'font.size' : 20, 'lines.linewidth' : 3.5, 'figure.figsize' : (13,7)})

**Discretization**

In [ ]:
# Asset grid
def asset_grid(amin,amax,na):
  ubar = np.log(1.0 + np.log(1.0 + amax - amin))
  ugrid = np.linspace(0.0, ubar, na)
  return amin + np.exp(np.log(ugrid) - 1.0) - 1.0

In [ ]:
# AR(1) process: Rouwenhorst
def rouwenhorst_Pi(N, p):
    # base case Pi_2
    Pi = np.array([[p, 1 - p],
                   [1 - p, p]])

    # recursion to build up from Pi_2 to Pi_N
    for n in range(3, N + 1):
        Pi_old = Pi
        Pi = np.zeros((n, n))

        Pi[:-1, :-1] += p * Pi_old
        Pi[:-1, 1:] += (1 - p) * Pi_old
        Pi[1:, :-1] += (1 - p) * Pi_old
        Pi[1:, 1:] += p * Pi_old
        Pi[1:-1, :] /= 2

    return Pi

def stationary_markov(Pi, tol=1E-14):
    # start with uniform distribution over all states
    n = Pi.shape[0]
    pi = np.full(n, 1/n)

    # update distribution using Pi until successive iterations differ by less than tol
    for _ in range(10_000):
        pi_new = Pi.T @ pi
        if np.max(np.abs(pi_new - pi)) < tol:
            return pi_new
        pi = pi_new

def discretize_income(rho, sigma, n_e):
    # choose inner-switching probability p to match persistence rho
    p = (1+rho)/2

    # start with states from 0 to n_e-1, scale by alpha to match standard deviation sigma
    e = np.arange(n_e)
    alpha = 2*sigma/np.sqrt(n_e-1)
    e = alpha*e

    # obtain Markov transition matrix Pi and its stationary distribution
    Pi = rouwenhorst_Pi(n_e, p)
    pi = stationary_markov(Pi)

    # e is log income, get income y and scale so that mean is 1
    y = np.exp(e)
    y /= np.vdot(pi, y)

    return y, pi, Pi

Calibration & Setup Grids, Value & Policy functions

In [ ]:
amin=0.0
amax=200.0
n_a=10_000
rho=0.9
sigma=0.1
n_e=21

# Asset grid
agrid=asset_grid(amin,amax,n_a)

# Productivity grid
ygrid, pi, Pi=discretize_income(rho, sigma, n_e)

# Value functions & Policy functions
V, NewV, gc, gn = np.zeros((n_a, n_e)), np.zeros((n_a, n_e)), np.zeros((n_a, n_e)), np.zeros((n_a, n_e))

ygrid, pi, Pi


<ipython-input-15-e8c278a17d29>:5: RuntimeWarning: divide by zero encountered in log
  return amin + np.exp(np.log(ugrid) - 1.0) - 1.0


(array([0.63621853, 0.66531689, 0.69574612, 0.72756706, 0.76084339,
        0.79564165, 0.83203146, 0.87008562, 0.90988023, 0.95149491,
        0.99501289, 1.04052124, 1.08811097, 1.13787728, 1.18991973,
        1.24434242, 1.30125421, 1.36076894, 1.42300565, 1.48808886,
        1.55614874]),
 array([9.53674316e-07, 1.90734863e-05, 1.81198120e-04, 1.08718872e-03,
        4.62055206e-03, 1.47857666e-02, 3.69644165e-02, 7.39288330e-02,
        1.20134354e-01, 1.60179138e-01, 1.76197052e-01, 1.60179138e-01,
        1.20134354e-01, 7.39288330e-02, 3.69644165e-02, 1.47857666e-02,
        4.62055206e-03, 1.08718872e-03, 1.81198120e-04, 1.90734863e-05,
        9.53674316e-07]),
 array([[3.58485922e-01, 3.77353603e-01, 1.88676801e-01, 5.95821478e-02,
         1.33275857e-02, 2.24464601e-03, 2.95348159e-04, 3.10892799e-05,
         2.65895157e-06, 1.86593093e-07, 1.08027580e-08, 5.16878373e-10,
         2.04030937e-11, 6.60828945e-13, 1.73902354e-14, 3.66110219e-16,
         6.02154965e-18, 7.4